<a href="https://colab.research.google.com/github/avi26-git/TensforFlow/blob/master/GettingStarted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Starting with TensorFlow:



Learning Objectives:

Learn fundamental TensorFlow concepts
Use the LinearRegressor class in TensorFlow to predict median housing price, at the granularity of city blocks, based on one input feature
Evaluate the accuracy of a model's predictions using Root Mean Squared Error (RMSE)
Improve the accuracy of a model by tuning its hyperparameters

Loading the necessary libraries

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

This housing data is based on 1990 census data from California.

# Getting and setting data

In [0]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

This data has to be randomized just to be sure not to get any pathological ordering effects that might harm the performance of Stochastic Gradient Descent.
Also, we'll scale median_house_value to be in units of thousands, so it can be learned a little more easily with learning rates in a range that we usually use.

In [4]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
12277,-121.5,39.5,25.0,848.0,153.0,436.0,155.0,3.9,93.8
8671,-118.5,34.0,39.0,2603.0,456.0,928.0,410.0,7.9,500.0
2632,-117.7,33.6,16.0,4277.0,565.0,1642.0,549.0,8.0,286.6
15467,-122.3,37.6,52.0,2351.0,494.0,1126.0,482.0,4.0,356.9
5175,-118.1,33.9,19.0,1793.0,447.0,1222.0,452.0,2.7,195.8
...,...,...,...,...,...,...,...,...,...
8840,-118.7,34.3,24.0,2983.0,406.0,1203.0,381.0,6.3,302.0
13054,-121.9,37.6,13.0,5519.0,833.0,2444.0,825.0,7.1,393.2
15909,-122.4,37.8,52.0,2520.0,649.0,959.0,607.0,5.8,500.0
15702,-122.4,37.8,52.0,157.0,28.0,88.0,27.0,3.7,162.5


Examining the data would be the next step.

In [5]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


#Build a model

Let's try to predict median_house_value, which will be our label. We'll use total_rooms as our input feature.
Using the TF Estimator API.